# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [89]:
import pandas as pd
import numpy as np

columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('C:\\Users\\Lisa\\production\\05_src\\data\\adult\\adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

df_raw = adult_dt

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [90]:
from sklearn.model_selection import train_test_split

# Create the features DataFrame (X) and target DataFrame (Y)
x = adult_dt.drop('income', axis=1) 
y = adult_dt['income']               

# Convert categorical variables into dummy/indicator variables
x = pd.get_dummies(x, drop_first=True)

# Obtain the training and testing datasets with a 70-30% split
x_train, x_test, Y_train, Y_test = train_test_split(x, Y, test_size=0.3, random_state=42)

# Output the shapes of the resulting datasets
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

x_train shape: (22792, 100)
x_test shape: (9769, 100)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 

+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)? 


Q1. Summary based on https://scikit-learn.org/ and from an article on Medium "What is random_state? (https://kishanmodasiya.medium.com/what-the-heck-is-random-state-24a7a8389f3d):
it controls for randomness by having shuffling applied to the data before applying the split (on train_test_split for splitting data into training and testing dataset). Whenever randomization is part of a Scikit-learn algorithm, a random_state parameter may be provided to control the random number generator used. Popular integer random seeds is 42.

Q2. Summary based on https://scikit-learn.org/ and from an article on Medium "What is random_state? (https://kishanmodasiya.medium.com/what-the-heck-is-random-state-24a7a8389f3d):
1. If we just split data without shuffling, it will give good performance on the training dataset but poor performance on testing. For that, we need to do shuffling of data. We use random_state to avoid this problem.
2. When we split data, we want consistence in the result of the dataset every time. This means if we rerun the code, then the training and testing dataset will remain the same every time. For that random_state is required.
3. For different random_state we can see the difference in performance (error score might be minimized for that random_state.)

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [91]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline

# Define the numerical and categorical features
num_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define numerical transformations
num_transforms = Pipeline(steps=[
    ('imputer', KNNImputer()),  
    ('scaler', RobustScaler())
])

# Define categorical transformations
cat_transforms = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# Combine transformations into ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transforms, num_features),
        ('cat_transforms', cat_transforms, cat_features)
    ]
)

# Create the main pipeline
pipe_simple = Pipeline([
    ('preprocessor', preprocessor)
])

pipe_simple

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])]))])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [92]:
from sklearn.ensemble import RandomForestClassifier

# Define the numerical and categorical features
num_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define numerical transformations
num_transforms = Pipeline(steps=[
    ('imputer', KNNImputer()),  
    ('scaler', RobustScaler())
])

# Define categorical transformations
cat_transforms = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# Combine transformations into ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transforms, num_features),
        ('cat_transforms', cat_transforms, cat_features)
    ]
)

# Create the main pipeline and add the classifier
pipe_simple = Pipeline([
    ('preprocessing', preprocessor),  # Preprocessing step
    ('classifier', RandomForestClassifier(random_state=42))  # Classifier step
])

pipe_simple

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [93]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Define the numerical and categorical features
num_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define numerical transformations
num_transforms = Pipeline(steps=[
    ('imputer', KNNImputer()),  
    ('scaler', RobustScaler())
])

# Define categorical transformations
cat_transforms = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# Combine transformations into ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transforms, num_features),
        ('cat_transforms', cat_transforms, cat_features)
    ]
)

# Create the main pipeline and add the classifier
pipe_simple = Pipeline([
    ('preprocessing', preprocessor),  # Preprocessing step
    ('classifier', RandomForestClassifier(random_state=42))  # Classifier step
])

# Prepare features and target variable
df_raw = adult_dt
x = df_raw.drop(columns='income')
y = df_raw['income']

# Check for NaN values in the raw data
print("NaN values in raw data:")
print(df_raw.isnull().sum())

# Prepare features and target variable
x = df_raw.drop(columns='income')
y = df_raw['income']

# Check for NaN values in the target variable
print("NaN values in target variable:")
print(y.isnull().sum())

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")

# Define the scoring metrics
scoring = {
    'neg_log_loss': make_scorer(log_loss, needs_proba=True),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True),
    'accuracy': make_scorer(accuracy_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}

# Perform cross-validation
results = cross_validate(pipe_simple, x, y, scoring=scoring, cv=5, return_train_score=True)

# Convert results to a DataFrame for easier reporting
results = cross_validate(pipe_simple, x, y, scoring=scoring, cv=5, return_train_score=True)
results_df = pd.DataFrame(results)
print("Cross-validation results:")
print(results_df.mean())

# Fit the model on the training set
pipe_simple.fit(x_train, y_train)

# Evaluate the model on the test set
test_score = pipe_simple.score(x_test, y_test)
print(f"Test set accuracy: {test_score}")

# Display results as a DataFrame for training and validation metrics
res_simple_dict = cross_validate(pipe_simple, x_train, y_train, cv=5, scoring=scoring)
res_simple = pd.DataFrame(res_simple_dict).assign(experiment=1)
print(res_simple)


NaN values in raw data:
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64
NaN values in target variable:
0
x_train shape: (26048, 14), y_train shape: (26048,)


c:\Users\Lisa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
c:\Users\Lisa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Lisa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Lisa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transf

Cross-validation results:
fit_time                   26.656397
score_time                  0.447875
test_neg_log_loss           0.364890
train_neg_log_loss          0.080325
test_roc_auc                0.903114
train_roc_auc               1.000000
test_accuracy               0.854212
train_accuracy              0.999977
test_balanced_accuracy      0.774887
train_balanced_accuracy     0.999985
dtype: float64
Test set accuracy: 0.8582834331337326


c:\Users\Lisa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Lisa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


    fit_time  score_time  test_neg_log_loss  test_roc_auc  test_accuracy  \
0  19.441233    0.519731           0.391983      0.897204       0.842610   
1  18.931695    0.389062           0.344858      0.905487       0.848369   
2  19.200308    0.338153           0.378132      0.902404       0.848369   
3  19.093467    0.346859           0.363162      0.906481       0.858898   
4  19.291050    0.336475           0.375729      0.907276       0.860242   

   test_balanced_accuracy  experiment  
0                0.757742           1  
1                0.774061           1  
2                0.764529           1  
3                0.780732           1  
4                0.782706           1  


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [94]:
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Define the scoring metrics
scoring = {
    'neg_log_loss': make_scorer(log_loss, needs_proba=True),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True),
    'accuracy': make_scorer(accuracy_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}

# Perform cross-validation
results = cross_validate(pipe_simple, x, y, scoring=scoring, cv=5, return_train_score=True)

# Convert results to a DataFrame for easier manipulation
results_df = pd.DataFrame(results)

# Display fold-level results for validation scores
# Select relevant columns
fold_results = results_df[['test_neg_log_loss', 'test_roc_auc', 'test_accuracy', 'test_balanced_accuracy']]

# Sort by negative log loss
sorted_fold_results = fold_results.sort_values(by='test_neg_log_loss', ascending=True)

# Print the sorted results
print("Fold-level results sorted by negative log loss:")
print(sorted_fold_results)

c:\Users\Lisa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
c:\Users\Lisa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Lisa\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Fold-level results sorted by negative log loss:
   test_neg_log_loss  test_roc_auc  test_accuracy  test_balanced_accuracy
2           0.348129      0.906473       0.852733                0.778899
4           0.350558      0.907448       0.857647                0.780176
3           0.358425      0.901815       0.857955                0.777983
0           0.362914      0.902940       0.853984                0.770252
1           0.404425      0.896894       0.848741                0.767124


Calculate the mean of each metric. 

In [95]:
# Calculate the mean of each metric
mean_metrics = fold_results.mean()

# Print the mean values of each metric
print("Mean of each metric from cross-validation:")
print(mean_metrics)

Mean of each metric from cross-validation:
test_neg_log_loss         0.364890
test_roc_auc              0.903114
test_accuracy             0.854212
test_balanced_accuracy    0.774887
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [96]:
import numpy as np
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Get predicted probabilities for the test set
y_pred_proba = pipe_simple.predict_proba(x_test)

# Calculate predictions for accuracy and balanced accuracy
y_pred = pipe_simple.predict(x_test)

# Calculate each metric
neg_log_loss_value = log_loss(y_test, y_pred_proba)
roc_auc_value = roc_auc_score(y_test, y_pred_proba[:, 1])  # Use the probabilities for the positive class
accuracy_value = accuracy_score(y_test, y_pred)
balanced_accuracy_value = balanced_accuracy_score(y_test, y_pred)

# Store results in a dictionary
results_dict = {
    'negative_log_loss': neg_log_loss_value,
    'roc_auc': roc_auc_value,
    'accuracy': accuracy_value,
    'balanced_accuracy': balanced_accuracy_value
}

# Display the results
print("Performance metrics on the testing data:")
print(results_dict)



Performance metrics on the testing data:
{'negative_log_loss': 0.36914261557963174, 'roc_auc': 0.9073497639454077, 'accuracy': 0.8582834331337326, 'balanced_accuracy': 0.7802674615611108}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Once the target variable is recoded, it can be easily used in various functions and methods throughout your analysis. Converting the income variable to a binary format (0 for '<=50K' and 1 for '>50K') simplifies subsequent analysis and model training.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.